# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [2]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [3]:
from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(
    data, 
    test_size=0.2,
    random_state=42,
    stratify=data['label']
)

## Data Preprocessing

In [4]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [5]:
import re
from bs4 import BeautifulSoup, Comment

def clean_html_text(text):

    if pd.isna(text) or not isinstance(text, str):
        return ""
    
    # 1.
    text = re.sub(r'<script[^>]*>.*?</script>', '', text, flags=re.DOTALL|re.IGNORECASE)
    text = re.sub(r'<style[^>]*>.*?</style>', '', text, flags=re.DOTALL|re.IGNORECASE)
    
    # 2.
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    
    # 3.
    soup = BeautifulSoup(text, 'html.parser')
    text = soup.get_text(separator=' ')
    
    # 4.
    text = re.sub(r'[^\x00-\x7F]', '', text)
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()


data_train['cleaned_text'] = data_train['text'].apply(clean_html_text)
data_val['cleaned_text'] = data_val['text'].apply(clean_html_text)

display(data_train['cleaned_text'])

C:\Users\yagoc\AppData\Local\Temp\ipykernel_7504\3094366305.py:17: MarkupResemblesLocatorWarning: The input passed in on this line looks more like a URL than HTML or XML.

If you meant to use Beautiful Soup to parse the web page found at a certain URL, then something has gone wrong. You should use an Python package like 'requests' to fetch the content behind the URL. Once you have the content as a string, you can feed that string into Beautiful Soup.

However, if you want to parse some data that happens to look like a URL, then nothing has gone wrong: you are using Beautiful Soup correctly, and this warning is spurious and can be filtered. To make this warning go away, run this code before calling the BeautifulSoup constructor:

    from bs4 import MarkupResemblesLocatorWarning
    import warnings

    warnings.filterwarnings("ignore", category=MarkupResemblesLocatorWarning)
    
  soup = BeautifulSoup(text, 'html.parser')


442    Dear=2C Good day hope fine=2Cdear am writting ...
962    FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...
971                                             Will do.
190    FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND ...
551    Dear Friend, My name is LOI C.ESTRADA,The wife...
                             ...                        
625    http://www.rte.ieews/2000/0418/zimbabwe.html I...
53     IV> Dear, It is my pleasure to write you after...
728    Will do. Think we are in good shape. Immelt In...
664    With your permission we are changing the state...
798                      Yes we will. Thank you Courtney
Name: cleaned_text, Length: 800, dtype: object

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [6]:
def preprocess_text(text):
    
    if pd.isna(text) or not isinstance(text, str):
        return ""
    
    # Remove all special characters (keep only letters and spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    
    # Remove single characters from anywhere in the text
    text = re.sub(r'\b[a-zA-Z]\b', '', text)
    
    # Remove single characters from the start of the string
    text = re.sub(r'^\w\s+', '', text)
    
    # Substitute multiple spaces with single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove prefixed 'b' (common when converting bytes to string)
    text = re.sub(r'^b\s+', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove leading and trailing spaces
    text = text.strip()
    
    return text


data_train['preprocessed_text'] = data_train['cleaned_text'].apply(preprocess_text)
data_val['preprocessed_text'] = data_val['cleaned_text'].apply(preprocess_text)

display(data_train['preprocessed_text'])

442    dearc good day hope finecdear am writting this...
962    from mr henry kaborethe chief auditor incharge...
971                                              will do
190    from the desk of dradamu ismalerauditing and a...
551    dear friend my name is loi cestradathe wife of...
                             ...                        
625    httpwwwrteieewszimbabwehtml got yourcontact th...
53     iv dear it is my pleasure to write you after m...
728    will do think we are in good shape immelt indr...
664    with your permission we are changing the state...
798                       yes we will thank you courtney
Name: preprocessed_text, Length: 800, dtype: object

## Now let's work on removing stopwords
Remove the stopwords.

In [7]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    # split text into words
    words = text.split()

    # filter stopwords
    filtered_words = [word for word in words if word not in stop_words]

    # return joined string
    return ' '.join(filtered_words)

data_train['stopwords_text'] = data_train['preprocessed_text'].apply(remove_stopwords)
data_val['stopwords_text'] = data_val['preprocessed_text'].apply(remove_stopwords)

display(data_train['stopwords_text'])

442    dearc good day hope finecdear writting mail du...
962    mr henry kaborethe chief auditor inchargeforei...
971                                                     
190    desk dradamu ismalerauditing accounting manage...
551    dear friend name loi cestradathe wife mr josep...
                             ...                        
625    httpwwwrteieewszimbabwehtml got yourcontact se...
53     iv dear pleasure write much consideration sinc...
728    think good shape immelt indra huntsman making ...
664    permission changing statement spokesmans name ...
798                                   yes thank courtney
Name: stopwords_text, Length: 800, dtype: object

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [8]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk

# Download required NLTK data
nltk.download('averaged_perceptron_tagger_eng', quiet=True)

wordnet_lemma = WordNetLemmatizer()

def get_wordnet_pos(word):
    
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {
        "J": wordnet.ADJ,
        "N": wordnet.NOUN,
        "V": wordnet.VERB,
        "R": wordnet.ADV
    }
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(text):
    # split into words
    words = text.split()

    # lemmatize each word
    lemmatized = [wordnet_lemma.lemmatize(word, get_wordnet_pos(word)) for word in words]

    # return lemmatized string
    return ' '. join(lemmatized)

data_train['lemma_text'] = data_train['stopwords_text'].apply(lemmatize_text)
data_val['lemma_text'] = data_val['stopwords_text'].apply(lemmatize_text)

display(data_train['lemma_text'])

442    dearc good day hope finecdear writting mail du...
962    mr henry kaborethe chief auditor inchargeforei...
971                                                     
190    desk dradamu ismalerauditing accounting manage...
551    dear friend name loi cestradathe wife mr josep...
                             ...                        
625    httpwwwrteieewszimbabwehtml get yourcontact se...
53     iv dear pleasure write much consideration sinc...
728    think good shape immelt indra huntsman make ca...
664    permission change statement spokesman name rat...
798                                   yes thank courtney
Name: lemma_text, Length: 800, dtype: object

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [9]:
from collections import Counter

# separate spam and ham
spam_text = ' '.join(data_train[data_train['label'] == 1]['lemma_text'])
ham_text = ' '.join(data_train[data_train['label'] == 0]['lemma_text'])

# count words 
spam_words = Counter(spam_text.split())
ham_words = Counter(ham_text.split())

# get top 10
print(f"Top 10 spam words: {spam_words.most_common(10)}")

print(f"Top 10 ham words: {ham_words.most_common(10)}")

Top 10 spam words: [('money', 713), ('account', 602), ('bank', 572), ('fund', 547), ('u', 444), ('transfer', 401), ('business', 386), ('contact', 357), ('transaction', 333), ('country', 324)]
Top 10 ham words: [('work', 94), ('call', 92), ('say', 92), ('would', 91), ('president', 90), ('u', 88), ('get', 76), ('percent', 76), ('state', 72), ('mr', 70)]


## Extra features

In [10]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['lemma_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['lemma_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['lemma_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['lemma_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['lemma_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['lemma_text'].apply(lambda x: len(x)) 

data_train.head(25)

,text,label,cleaned_text,preprocessed_text,stopwords_text,lemma_text,money_mark,suspicious_words,text_len
442,Dear=2C Good day hope fine=2Cdear am writting ...,1,Dear=2C Good day hope fine=2Cdear am writting ...,dearc good day hope finecdear am writting this...,dearc good day hope finecdear writting mail du...,dearc good day hope finecdear writting mail du...,1,1,1005
962,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,1,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...,from mr henry kaborethe chief auditor incharge...,mr henry kaborethe chief auditor inchargeforei...,mr henry kaborethe chief auditor inchargeforei...,0,1,1920
971,Will do.,0,Will do.,will do,,,0,0,0
190,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...,1,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND ...,from the desk of dradamu ismalerauditing and a...,desk dradamu ismalerauditing accounting manage...,desk dradamu ismalerauditing accounting manage...,1,1,382
551,"Dear Friend, My name is LOI C.ESTRADA,The wife...",1,"Dear Friend, My name is LOI C.ESTRADA,The wife...",dear friend my name is loi cestradathe wife of...,dear friend name loi cestradathe wife mr josep...,dear friend name loi cestradathe wife mr josep...,1,1,1484
495,FROM =3AMR=2ECOLLINCE ADDOATTN =3AI NEED YOUR ...,1,FROM =3AMR=2ECOLLINCE ADDOATTN =3AI NEED YOUR ...,from amrecollince addoattn ai need your urgent...,amrecollince addoattn ai need urgent assistanc...,amrecollince addoattn ai need urgent assistanc...,1,1,1199
845,Pls send me call sheet for Equadoran--i don't ...,0,Pls send me call sheet for Equadoran--i don't ...,pls send me call sheet for equadorani dont hav...,pls send call sheet equadorani dont clue,pls send call sheet equadorani dont clue,0,0,40
821,Yes I will arrange.,0,Yes I will arrange.,yes will arrange,yes arrange,yes arrange,0,0,11
409,FYI Ã¢ÂÂ I will limit Haiti emails to major ...,0,FYI I will limit Haiti emails to major fyi inf...,fyi will limit haiti emails to major fyi infor...,fyi limit haiti emails major fyi information,fyi limit haiti email major fyi information,0,0,43
794,Mr=2E Zuhair Idris Jordan Kuwait BankCredit De...,1,Mr=2E Zuhair Idris Jordan Kuwait BankCredit De...,mre zuhair idris jordan kuwait bankcredit depa...,mre zuhair idris jordan kuwait bankcredit depa...,mre zuhair idris jordan kuwait bankcredit depa...,1,1,1830


## How would work the Bag of Words with Count Vectorizer concept?

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# Create vectorizer 
count_vec = CountVectorizer(max_features=1000, ngram_range=(1,2))

# fit on training data and transform both
X_train_counts = count_vec.fit_transform(data_train['lemma_text'])
X_val_counts = count_vec.transform(data_val['lemma_text'])

print(f'Training shape: {X_train_counts.shape}')
print(f'Validation shape: {X_val_counts.shape}')

Training shape: (800, 1000)
Validation shape: (200, 1000)


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [15]:
# TfidVectorizer 

tfidf = TfidfVectorizer(max_features=1000, ngram_range=(1,2))

# Fit on training, transform both
X_train_tfidf = tfidf.fit_transform(data_train['lemma_text'])
X_val_tfidf = tfidf.transform(data_val['lemma_text'])

print(f'Training shape: {X_train_tfidf.shape}')
print(f'Validation shape: {X_val_tfidf.shape}')


Training shape: (800, 1000)
Validation shape: (200, 1000)


## And the Train a Classifier?

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Create the classifier 
clf = MultinomialNB()

clf.fit(X_train_tfidf, data_train['label'])

# predict
y_pred = clf.predict(X_val_tfidf)

# evaluate
print(f"Accuracy: {accuracy_score(data_val['label'], y_pred):.4f}")
print("\nClassification Report: ")
print(classification_report(data_val['label'], y_pred, target_names=['Ham', 'Spam']))

Accuracy: 0.9500

Classification Report: 
              precision    recall  f1-score   support

         Ham       0.97      0.94      0.95       112
        Spam       0.92      0.97      0.94        88

    accuracy                           0.95       200
   macro avg       0.95      0.95      0.95       200
weighted avg       0.95      0.95      0.95       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [14]:
# Your code